In [ ]:
from pathlib import Path
from PIL import Image
import torch

from src.model import load_clip

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

#загружаем модель
model, processor = load_clip(device)

Простой пример инференса: 

In [ ]:
DATA_DIR = Path("data")

image_paths = sorted(list(DATA_DIR.glob("*.jpg")) + list(DATA_DIR.glob("*.png")))
print("Found images:")
for p in image_paths:
    print(" -", p)

image_paths

In [ ]:
texts = [
    "a magnifying glass",
    "a cute dog",
    "a smiling person",
]

texts

In [ ]:
from torch.nn.functional import softmax

for img_path in image_paths:
    img = Image.open(img_path).convert("RGB")

    inputs = processor(
        text=texts,
        images=img,
        return_tensors="pt",
        padding=True
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    logits_per_image = outputs.logits_per_image  
    logits = logits_per_image[0]              

    probs = softmax(logits, dim=0)  # shape [num_texts]

    print(f"\nResults for {img_path.name}:")
    for text, p in zip(texts, probs):
        print(f"  {text:25s} {float(p):.3f}")
